In [2]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
# !pip install selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
from pandas import DataFrame
# import pyautogui
from requests.exceptions import ConnectionError

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
internal_urls = set()
external_urls = set()

In [5]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [6]:
#The function gives all urls
count = 0
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        global count
        count = count+1
        print(f"{count}[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

In [7]:
total_urls_visited = 0
def crawl(url, max_urls=30000):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except Exception as e:
        print(e)

In [8]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.archanaskitchen.com/"
internal_urls.add(base_url)

In [9]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.archanaskitchen.com', path='/', params='', query='', fragment='')
https://www.archanaskitchen.com
www.archanaskitchen.com/
www.archanaskitchen.com


In [ ]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

In [ ]:
len(internal_urls)

In [29]:
setInternalUrls = set(internal_urls)
len(setInternalUrls)

22235

In [30]:
df = DataFrame(internal_urls,columns=['URL'])

In [31]:
df.shape

(22235, 1)

In [32]:
dfrecipe = df[df['URL'].str.contains('recipe')]

In [33]:
dfrecipe

,URL
0,https://www.archanaskitchen.com/masala-karela-...
1,http://www.archanaskitchen.com/spicy-tomato-ri...
3,http://www.archanaskitchen.com/ragi-vermicelli...
4,http://www.archanaskitchen.com/tag/kids-lunch-...
5,http://www.archanaskitchen.com/gongura-chicken...
...,...
22226,https://www.archanaskitchen.com/kerala-cheriya...
22227,http://www.archanaskitchen.com/pav-bhaji-flavo...
22230,https://www.archanaskitchen.com/tag/chilli-rec...
22231,https://www.archanaskitchen.com/keerai-masiyal...


In [34]:
dfrecipe.to_csv('urls_recipe10k.csv')